In [64]:
import pretty_midi
import numpy as np
import os
import IPython
import fluidsynth
from ast import literal_eval

In [24]:
def bpm_to_120(midi_file):
    """ 
    This function evens out the tempo throuout the song to be 120 BPM,
    even if there are tempo changes. It gets a midi file as an input
    and outputs a modified pretty_midi object
    """
    
    mid = pretty_midi.PrettyMIDI(midi_file)
    
    tempo = mid.get_tempo_changes()
    num_of_changes = len(tempo[0])
    full_length = mid.get_end_time()

    old_times = []
    changes = [0]
    
    for i in range(num_of_changes):
        old_times.append(tempo[0][i])
        if i < (num_of_changes - 1):
            changes.append((tempo[0][i+1] - tempo[0][i]) * (tempo[1][i]/120))
        else:
            changes.append((full_length - tempo[0][i]) * (tempo[1][i]/120))
        
    old_times.append(full_length)
    new_times = np.cumsum(changes)

    mid.adjust_times(old_times, new_times)

    song_title = os.path.splitext(os.path.basename(midi_file))[0]
    
    return mid, song_title

In [46]:
mid_120, title = bpm_to_120('raw_data/Another-One-Bites-The-Dust-1.mid')

In [47]:
title

'Another-One-Bites-The-Dust-1'

In [48]:
def extract_guitar_and_drums(mid, song_title='unknown'):
    """This function extracts the guitar and drum tracks from a midi file.
       The input is a path to a midi file (for example: 'raw_data/song_name.mid') in string format
       The output is a dictionary with the song name, guitar track and drum track"""

    if type(mid) == str:
        mid = pretty_midi.PrettyMIDI(mid)
    
    guitars = []
    lengths_guitar = []
    drums = []
    lengths_drums = []
    
    for instrument in mid.instruments:
        if instrument.is_drum:
            drums.append(instrument)
            lengths_drums.append(len(instrument.notes))

        if (instrument.program >= 25) and (instrument.program <= 31):
            guitars.append(instrument)
            lengths_guitar.append(len(instrument.notes))

    drum_track = drums[lengths_drums.index(max(lengths_drums))]
    guitar_track = guitars[lengths_guitar.index(max(lengths_guitar))]

    # song_title = os.path.splitext(os.path.basename(midi_file))[0]
    
    

        
    song_dict = {'title': song_title,
                 'down_beats': mid.get_downbeats(),
                 'guitar': guitar_track,
                 'drums': drum_track
                }
    return song_dict

In [49]:
original = pretty_midi.PrettyMIDI('raw_data/Another-One-Bites-The-Dust-1.mid')

In [50]:
original.get_downbeats()

array([  0.,   2.,   4.,   6.,   8.,  10.,  12.,  14.,  16.,  18.,  20.,
        22.,  24.,  26.,  28.,  30.,  32.,  34.,  36.,  38.,  40.,  42.,
        44.,  46.,  48.,  50.,  52.,  54.,  56.,  58.,  60.,  62.,  64.,
        66.,  68.,  70.,  72.,  74.,  76.,  78.,  80.,  82.,  84.,  86.,
        88.,  90.,  92.,  94.,  96.,  98., 100., 102., 104., 106., 108.,
       110., 112., 114., 116., 118., 120., 122., 124., 126., 128., 130.,
       132., 134., 136., 138., 140., 142., 144., 146., 148., 150., 152.,
       154., 156., 158., 160., 162., 164., 166., 168., 170., 172., 174.,
       176., 178.])

In [51]:
song_dict = extract_guitar_and_drums(mid_120, title)
song_dict

{'title': 'Another-One-Bites-The-Dust-1',
 'down_beats': array([  0.,   2.,   4.,   6.,   8.,  10.,  12.,  14.,  16.,  18.,  20.,
         22.,  24.,  26.,  28.,  30.,  32.,  34.,  36.,  38.,  40.,  42.,
         44.,  46.,  48.,  50.,  52.,  54.,  56.,  58.,  60.,  62.,  64.,
         66.,  68.,  70.,  72.,  74.,  76.,  78.,  80.,  82.,  84.,  86.,
         88.,  90.,  92.,  94.,  96.,  98., 100., 102., 104., 106., 108.,
        110., 112., 114., 116., 118., 120., 122., 124., 126., 128., 130.,
        132., 134., 136., 138., 140., 142., 144., 146., 148., 150., 152.,
        154., 156., 158., 160., 162., 164., 166., 168., 170., 172., 174.,
        176., 178.]),
 'guitar': Instrument(program=28, is_drum=False, name="+FUNK GTR"),
 'drums': Instrument(program=0, is_drum=True, name="DRUMS")}

In [52]:
def tracks_to_bars(song_dict: dict):
    """This function accepts a dictionary as an input with 4 keys: 'title', 'down_beats', 'guitar', 'drums'.
    The function takes the guitar and drums, both pretty_midi instrument objects, and cuts them up into a sequence of individual bars.
    The output is a dictionary that contains the following keys/values: song_title, a list of guitar bars, a list of drum bars, and a list of the song's downbeats
    """
    
    new_dict={}
    new_dict['song_title']=song_dict['title']
    guitar = song_dict['guitar']
    drums = song_dict['drums']
    down_beats_array = song_dict['down_beats']


    guitar_bars_list = []
    drums_bars_list = []

    for index, start_time in enumerate(down_beats_array):
        try:
            end_time = down_beats_array[index+1]            
            guitar_bar = []
            drums_bar = []
            for guitar_note, drum_note in zip(guitar.notes, drums.notes):
                if (guitar_note.start >= start_time) and (guitar_note.end < end_time):
                    guitar_bar.append(guitar_note)    
                if (drum_note.start >= start_time) and (drum_note.end < end_time):
                    drums_bar.append(drum_note)
            drums_bars_list.append(drums_bar)
            guitar_bars_list.append(guitar_bar)
        except:
            print("done")
            pass
    new_dict['guitar_bars'] = guitar_bars_list
    new_dict['drum_bars'] = drums_bars_list
    new_dict['down_beats'] = down_beats_array.tolist()
    return new_dict

In [53]:
new_dict = tracks_to_bars(song_dict)
new_dict

done


{'song_title': 'Another-One-Bites-The-Dust-1',
 'guitar_bars': [[],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [Note(start=62.000000, end=62.062500, pitch=71, velocity=47),
   Note(start=62.000000, end=62.067708, pitch=64, velocity=50),
   Note(start=62.000000, end=62.072917, pitch=67, velocity=53),
   Note(start=62.000000, end=62.072917, pitch=62, velocity=53),
   Note(start=62.125000, end=62.192708, pitch=62, velocity=96),
   Note(start=62.125000, end=62.197917, pitch=71, velocity=96),
   Note(start=62.125000, end=62.197917, pitch=67, velocity=96),
   Note(start=62.125000, end=62.208333, pitch=64, velocity=96),
   Note(start=62.250000, end=62.291667, pitch=62, velocity=24),
   Note(start=62.250000, end=62.296875, pitch=64, velocity=24),
   Note(start=62.375000, end=62.416667, pitch=62, velocity=35),
   Note(start=62.375000, end=62.427083, pitch=64

In [76]:
len(new_dict['down_beats'])

90

In [77]:
len(new_dict['guitar_bars'])

89

In [79]:
len(new_dict['drum_bars'])

89

In [84]:
def standardize_bars(list_of_bars, downbeats):
    """ 
    This function standardizes the timing of musical bars 
    so that each bar will start at the same time point.
    It gets a list of bars and the list of downbeats as inputs
    and returns a list of bars that all start with time = 1
    """
    
    for i in range(len(list_of_bars)):
        for j in range(len(list_of_bars[i])):
            if i == 0:
                list_of_bars[0][j].start = list_of_bars[0][j].start / downbeats[1]
                list_of_bars[0][j].end = list_of_bars[0][j].end / downbeats[1]

            list_of_bars[i][j].start = list_of_bars[i][j].start / downbeats[i]
            list_of_bars[i][j].end = list_of_bars[i][j].end / downbeats[i]

    return list_of_bars
    

In [85]:
standardized_guitar = standardize_bars(new_dict['guitar_bars'], new_dict['down_beats'])
standardized_guitar

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [Note(start=0.000004, end=0.000004, pitch=71, velocity=47),
  Note(start=0.000004, end=0.000004, pitch=64, velocity=50),
  Note(start=0.000004, end=0.000004, pitch=67, velocity=53),
  Note(start=0.000004, end=0.000004, pitch=62, velocity=53),
  Note(start=0.000004, end=0.000004, pitch=62, velocity=96),
  Note(start=0.000004, end=0.000004, pitch=71, velocity=96),
  Note(start=0.000004, end=0.000004, pitch=67, velocity=96),
  Note(start=0.000004, end=0.000004, pitch=64, velocity=96),
  Note(start=0.000004, end=0.000004, pitch=62, velocity=24),
  Note(start=0.000004, end=0.000004, pitch=64, velocity=24),
  Note(start=0.000004, end=0.000004, pitch=62, velocity=35),
  Note(start=0.000004, end=0.000004, pitch=64, velocity=35),
  Note(start=0.000004, end=0.000004, pitch=71, velocity=60),
  Note(start=0.000004, end=0.000004, pitch=67, veloc

In [86]:
len(standardized_guitar)

89

In [88]:
standardized_drums = standardize_bars(new_dict['drum_bars'], new_dict['down_beats'])
standardized_drums

[[],
 [],
 [Note(start=0.089844, end=0.089925, pitch=40, velocity=102),
  Note(start=0.091797, end=0.091878, pitch=40, velocity=122)],
 [Note(start=0.027778, end=0.027802, pitch=49, velocity=70),
  Note(start=0.027778, end=0.027802, pitch=35, velocity=109),
  Note(start=0.028935, end=0.028959, pitch=42, velocity=57),
  Note(start=0.030093, end=0.030117, pitch=42, velocity=85),
  Note(start=0.030093, end=0.030117, pitch=40, velocity=115),
  Note(start=0.030093, end=0.030117, pitch=35, velocity=112),
  Note(start=0.031250, end=0.031274, pitch=42, velocity=8),
  Note(start=0.032407, end=0.032432, pitch=42, velocity=90),
  Note(start=0.032407, end=0.032432, pitch=35, velocity=116),
  Note(start=0.033565, end=0.033589, pitch=42, velocity=26),
  Note(start=0.034722, end=0.034746, pitch=42, velocity=92),
  Note(start=0.034722, end=0.034746, pitch=40, velocity=117),
  Note(start=0.034722, end=0.034746, pitch=35, velocity=113),
  Note(start=0.035880, end=0.035904, pitch=42, velocity=4)],
 [Note

In [89]:
len(standardized_drums)

89

In [93]:
def mapping_dictionary(list_of_bars):
    bar_dict = {}
    for bar in list_of_bars:
        bar_dict[str(bar)] = bar

    return bar_dict

In [94]:
bar_dict = mapping_dictionary(standardized_drums)
bar_dict

{'[]': [],
 '[Note(start=0.089844, end=0.089925, pitch=40, velocity=102), Note(start=0.091797, end=0.091878, pitch=40, velocity=122)]': [Note(start=0.089844, end=0.089925, pitch=40, velocity=102),
  Note(start=0.091797, end=0.091878, pitch=40, velocity=122)],
 '[Note(start=0.027778, end=0.027802, pitch=49, velocity=70), Note(start=0.027778, end=0.027802, pitch=35, velocity=109), Note(start=0.028935, end=0.028959, pitch=42, velocity=57), Note(start=0.030093, end=0.030117, pitch=42, velocity=85), Note(start=0.030093, end=0.030117, pitch=40, velocity=115), Note(start=0.030093, end=0.030117, pitch=35, velocity=112), Note(start=0.031250, end=0.031274, pitch=42, velocity=8), Note(start=0.032407, end=0.032432, pitch=42, velocity=90), Note(start=0.032407, end=0.032432, pitch=35, velocity=116), Note(start=0.033565, end=0.033589, pitch=42, velocity=26), Note(start=0.034722, end=0.034746, pitch=42, velocity=92), Note(start=0.034722, end=0.034746, pitch=40, velocity=117), Note(start=0.034722, end=

In [101]:
bar_dict.keys()

dict_keys(['[]', '[Note(start=0.089844, end=0.089925, pitch=40, velocity=102), Note(start=0.091797, end=0.091878, pitch=40, velocity=122)]', '[Note(start=0.027778, end=0.027802, pitch=49, velocity=70), Note(start=0.027778, end=0.027802, pitch=35, velocity=109), Note(start=0.028935, end=0.028959, pitch=42, velocity=57), Note(start=0.030093, end=0.030117, pitch=42, velocity=85), Note(start=0.030093, end=0.030117, pitch=40, velocity=115), Note(start=0.030093, end=0.030117, pitch=35, velocity=112), Note(start=0.031250, end=0.031274, pitch=42, velocity=8), Note(start=0.032407, end=0.032432, pitch=42, velocity=90), Note(start=0.032407, end=0.032432, pitch=35, velocity=116), Note(start=0.033565, end=0.033589, pitch=42, velocity=26), Note(start=0.034722, end=0.034746, pitch=42, velocity=92), Note(start=0.034722, end=0.034746, pitch=40, velocity=117), Note(start=0.034722, end=0.034746, pitch=35, velocity=113), Note(start=0.035880, end=0.035904, pitch=42, velocity=4)]', '[Note(start=0.015625, en

In [95]:
len(bar_dict.keys())

36

In [109]:
bar_dict

{'[]': [],
 '[Note(start=0.089844, end=0.089925, pitch=40, velocity=102), Note(start=0.091797, end=0.091878, pitch=40, velocity=122)]': [Note(start=0.089844, end=0.089925, pitch=40, velocity=102),
  Note(start=0.091797, end=0.091878, pitch=40, velocity=122)],
 '[Note(start=0.027778, end=0.027802, pitch=49, velocity=70), Note(start=0.027778, end=0.027802, pitch=35, velocity=109), Note(start=0.028935, end=0.028959, pitch=42, velocity=57), Note(start=0.030093, end=0.030117, pitch=42, velocity=85), Note(start=0.030093, end=0.030117, pitch=40, velocity=115), Note(start=0.030093, end=0.030117, pitch=35, velocity=112), Note(start=0.031250, end=0.031274, pitch=42, velocity=8), Note(start=0.032407, end=0.032432, pitch=42, velocity=90), Note(start=0.032407, end=0.032432, pitch=35, velocity=116), Note(start=0.033565, end=0.033589, pitch=42, velocity=26), Note(start=0.034722, end=0.034746, pitch=42, velocity=92), Note(start=0.034722, end=0.034746, pitch=40, velocity=117), Note(start=0.034722, end=

In [96]:
def objects_to_strings(list_of_bars):
    list_of_strings = [str(bar) for bar in list_of_bars]
    return list_of_strings

In [97]:
strings = objects_to_strings(standardized_drums)
strings

['[]',
 '[]',
 '[Note(start=0.089844, end=0.089925, pitch=40, velocity=102), Note(start=0.091797, end=0.091878, pitch=40, velocity=122)]',
 '[Note(start=0.027778, end=0.027802, pitch=49, velocity=70), Note(start=0.027778, end=0.027802, pitch=35, velocity=109), Note(start=0.028935, end=0.028959, pitch=42, velocity=57), Note(start=0.030093, end=0.030117, pitch=42, velocity=85), Note(start=0.030093, end=0.030117, pitch=40, velocity=115), Note(start=0.030093, end=0.030117, pitch=35, velocity=112), Note(start=0.031250, end=0.031274, pitch=42, velocity=8), Note(start=0.032407, end=0.032432, pitch=42, velocity=90), Note(start=0.032407, end=0.032432, pitch=35, velocity=116), Note(start=0.033565, end=0.033589, pitch=42, velocity=26), Note(start=0.034722, end=0.034746, pitch=42, velocity=92), Note(start=0.034722, end=0.034746, pitch=40, velocity=117), Note(start=0.034722, end=0.034746, pitch=35, velocity=113), Note(start=0.035880, end=0.035904, pitch=42, velocity=4)]',
 '[Note(start=0.015625, en

In [115]:
len(strings)

89

In [100]:
from transformers import AutoModel, AutoTokenizer

/Users/Mark/.pyenv/versions/3.10.6/envs/BandIt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [ ]:
vocab_size = 36
sequence_length = 128

encoder = layers.TextVectorization(
    max_tokens = vocab_size,
    output_sequence_length = sequence_length,
    standardize = "lower_and_strip_punctuation",
    split = "whitespace",
    output_mode="int"
)

encoder.adapt(
    dataset_train_original.map(lambda text, label: text).batch(512)
)

In [ ]:
encoder.get_vocabulary()[:100]

In [102]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import WhitespaceSplit
from tokenizers.trainers import WordLevelTrainer
#from transformers import PreTrainedTokenizerFast
tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
#tokenizer.pre_tokenizer = WhitespaceSplit()
training_data = strings
trainer = WordLevelTrainer(
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)
    
#training_corpus = get_training_corpus()
tokenizer.train_from_iterator(training_data, trainer=trainer)

In [119]:
len(tokenizer.get_vocab())

41

SyntaxError: invalid syntax (1196025537.py, line 1)